# GET NEXT LINE
- Can use read, malloc, and free

## ~M~A~N~D~A~T~O~R~Y~

### get_next_line.h

In [ ]:
%%file ../gnl_working/get_next_line.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line.h                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:29 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:32 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef GET_NEXT_LINE_H
# define GET_NEXT_LINE_H
# ifndef BUFFER_SIZE
#  define BUFFER_SIZE 1024
# endif
# include <stdlib.h>
# include <unistd.h>
# include <stdint.h>

char	*get_next_line(int fd);
void	ft_bzero(void *str, size_t len);
void	*ft_memcpy(void *dest, const void *src, size_t len);
size_t	ft_strlen(const char *str);
char	*ft_strjoin(char const *s1, char const *s2);
char	*ft_strdup(const char *src);

#endif

Overwriting ../gnl_working/get_next_line.h


### get_next_line.c

In [13]:
%%file ../gnl_working/get_next_line.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:18 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:23 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line.h"

/*  *** nl_loc (newline locate) ***
 *
 *  Looks for a newline in the string and 
 *  returns a signed int indicating the index in the string where the 
 *  newline is located.
 *  returns -1 if newline not found or str is null.
 */

static int	nl_loc(char *str)
{
	size_t	i;

	if (str == 0)
		return (-1);
	i = 0;
	while (str[i] != '\n' && str[i] != '\0')
		i++;
	if (str[i] == 0)
		return (-1);
	return (i);
}

/*  *** str_trim (string trim) ***
 *
 *  str_trim will malloc memory of size 'len' + 2.
 *  Then, copy each character until a newline is found.
 *  Then, add a newline if one is found at the end of the string
 *  Then, add a null terminator and return the resulting string.
 */

static char	*str_trim(char *s_buf, size_t len)
{
	size_t	i;
	char	*ret_s_buf;

	i = 0;
	ret_s_buf = (char *)malloc(len + 2);
	if (!ret_s_buf)
		return (0);
	while (s_buf[i] != '\n' && s_buf[i])
	{
		ret_s_buf[i] = s_buf[i];
		i++;
	}
	if (s_buf[i] == '\n')
	{
		ret_s_buf[i] = s_buf[i];
		i++;
	}
	ret_s_buf[i] = 0;
	return (ret_s_buf);
}

/*  *** ret_str_build (Return String Build) ***
 *
 *  Takes the static buffer and cuts off portion after newline to
 *  make malloced string for return.
 */

static char	*ret_str_build(char *s_buf)
{
	size_t	nl_index;
	size_t	len_s_buf;
	char	*ret_s_buf;

	len_s_buf = ft_strlen(s_buf);
	if (len_s_buf <= 0)
		return (0);
	nl_index = nl_loc(s_buf);
	if ((int) nl_index == -1 || nl_index == len_s_buf - 1)
	{
		ret_s_buf = ft_strdup(s_buf);
		return (ret_s_buf);
	}
	ret_s_buf = str_trim(s_buf, nl_index);
	return (ret_s_buf);
}

/*  *** clean_s_buf (clean static buffer) ***
 *
 *  After a static buffer has been used to find an entire newline,
 *  clean_s_buf will remove an entire newline from the static buffer
 *  to allow carry over to next usage of get_next_line without needing
 *  to read from the file again.
 *  Returns address to new saved carry over on the heap.
 */

static char	*clean_s_buf(char *s_buf)
{
	size_t	nl_index;
	size_t	len_s_buf;
	char	*ret_s_buf;

	nl_index = nl_loc(s_buf);
	if ((int) nl_index == -1)
	{
		free(s_buf);
		return (0);
	}
	len_s_buf = ft_strlen(s_buf);
	if (len_s_buf - nl_index == 1)
	{
		free(s_buf);
		return (0);
	}
	ret_s_buf = ft_strdup(&(s_buf[nl_index + 1]));
	free(s_buf);
	return (ret_s_buf);
}

/*  *** get_next_line (get next line) ***
 *
 *  Uses static char buffer to keep portion after newline
 *  for next function call if needed.
 *  Checks for negative buffer size or invalid file descriptor.
 *  Returns malloced string from staring position to next newline
 *  in open file stream.
 */

char	*get_next_line(int fd)
{
	static char	*s_buf;
	char		*buf;
	char		*ret_str;
	int			r_size;

	if (fd < 0 || BUFFER_SIZE <= 0)
		return (0);
	buf = malloc(BUFFER_SIZE + 1);
	if (!buf)
		return (0);
	r_size = 1;
	while (nl_loc(s_buf) == -1)
	{
		r_size = read(fd, buf, BUFFER_SIZE);
		if (r_size <= 0)
			break ;
		buf[r_size] = 0;
		s_buf = ft_strjoin(s_buf, buf);
	}
	free(buf);
	if (r_size == -1)
		return (0);
	ret_str = ret_str_build(s_buf);
	s_buf = clean_s_buf(s_buf);
	return (ret_str);
}


Overwriting ../gnl_working/get_next_line.c


### get_next_line_utils.c

In [14]:
%%file ../gnl_working/get_next_line_utils.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line_utils.c                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:38 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:41 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line.h"

/*  *** ft_bzero (42 byte zero) ***
 *
 *  Sets memory values to 0 for initilization purposes.
 *  Returns nothing.
 */

void	ft_bzero(void *str, size_t len)
{
	size_t	i;

	i = 0;
	while (i < len)
	{
		*(char *)str = 0;
		str++;
		i++;
	}
}

/*  *** ft_memcpy (42 memory copy) ***
 *
 *  Copies len number of bytes starting from src to dest
 *  Returns pointer to destination string.
 */

void	*ft_memcpy(void *dest, const void *src, size_t len)
{
	size_t	i;

	i = 0;
	if (dest == 0 && src == 0)
		return (0);
	while (i < len)
	{
		*(char *)(dest + i) = *(char *)(src + i);
		i++;
	}
	return (dest);
}

/*  *** ft_strlen (42 string lenght) ***
 *
 *  Returns number of bytes from str address to next null terminator.
 */

size_t	ft_strlen(const char *str)
{
	size_t	i;

	i = 0;
	while (str && str[i])
		i++;
	return (i);
}

/*  *** ft_strjoin (42 string join) ***
 *
 *  Joins 2 string together to make a new string.
 *  Returns new malloced string 'ptr' that contains s1 followed by s2.
 */

char	*ft_strjoin(char const *s1, char const *s2)
{
	char	*ptr;
	size_t	s1_len;
	size_t	s2_len;

	s1_len = ft_strlen(s1);
	s2_len = ft_strlen(s2);
	if (s1_len + s2_len == 0)
		return (0);
	ptr = (char *)malloc((s1_len + s2_len + 1) * sizeof(char));
	if (ptr == 0)
		return (0);
	ft_bzero(ptr, (s1_len + s2_len + 1));
	ft_memcpy(ptr, (char *)s1, s1_len);
	ft_memcpy((ptr + s1_len), (char *)s2, s2_len);
	free((char *)s1);
	return (ptr);
}

/*  *** ft_strdup (42 string duplicate) ***
 *
 *  Creates a new string that is an exact copy of 'str'.
 *  Returns new malloced string that is an exact copy of 'str'.
 */

char	*ft_strdup(const char *src)
{
	char	*str_ret;
	size_t	src_len;
	size_t	i;

	i = 0;
	src_len = ft_strlen(src);
	str_ret = (char *)malloc((src_len + 1) * sizeof(char));
	if (!str_ret)
		return (0);
	while (src[i])
	{
		str_ret[i] = src[i];
		i++;
	}
	str_ret[i] = '\0';
	return (str_ret);
}

Overwriting ../gnl_working/get_next_line_utils.c


## ~B~O~N~U~S~

### get_next_line_bonus.h

In [6]:
%%file ../gnl_working/get_next_line_bonus.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line_bonus.h                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:29 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/30 18:02:06 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef GET_NEXT_LINE_BONUS_H
# define GET_NEXT_LINE_BONUS_H
# ifndef BUFFER_SIZE
#  define BUFFER_SIZE 1024
# endif
# include <stdlib.h>
# include <unistd.h>
# include <stdint.h>

char	*get_next_line(int fd);
void	ft_bzero(void *str, size_t len);
void	*ft_memcpy(void *dest, const void *src, size_t len);
size_t	ft_strlen(const char *str);
char	*ft_strjoin(char const *s1, char const *s2);
char	*ft_strdup(const char *src);

#endif


Overwriting ../gnl_working/get_next_line_bonus.h


### get_next_line_bonus.c

In [12]:
%%file ../gnl_working/get_next_line_bonus.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line_bonus.c                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:18 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/29 21:37:45 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line_bonus.h"

/*  *** nl_loc (newline locate) ***
 *
 *  Looks for a newline in the string and 
 *  returns a signed int indicating the index in the string where the 
 *  newline is located.
 *  returns -1 if newline not found or str is null.
 */

static int	nl_loc(char *str)
{
	size_t	i;

	if (str == 0)
		return (-1);
	i = 0;
	while (str[i] != '\n' && str[i] != '\0')
		i++;
	if (str[i] == 0)
		return (-1);
	return (i);
}

/*  *** str_trim (string trim) ***
 *
 *  str_trim will malloc memory of size 'len' + 2.
 *  Then, copy each character until a newline is found.
 *  Then, add a newline if one is found at the end of the string
 *  Then, add a null terminator and return the resulting string.
 */

static char	*str_trim(char *s_buf, size_t len)
{
	size_t	i;
	char	*ret_s_buf;

	i = 0;
	ret_s_buf = (char *)malloc(len + 2);
	if (!ret_s_buf)
		return (0);
	while (s_buf[i] != '\n' && s_buf[i])
	{
		ret_s_buf[i] = s_buf[i];
		i++;
	}
	if (s_buf[i] == '\n')
	{
		ret_s_buf[i] = s_buf[i];
		i++;
	}
	ret_s_buf[i] = 0;
	return (ret_s_buf);
}

/*  *** ret_str_build (Return String Build) ***
 *
 *  Takes the static buffer and cuts off portion after newline to
 *  make malloced string for return.
 */

static char	*ret_str_build(char *s_buf)
{
	size_t	nl_index;
	size_t	len_s_buf;
	char	*ret_s_buf;

	len_s_buf = ft_strlen(s_buf);
	if (len_s_buf <= 0)
		return (0);
	nl_index = nl_loc(s_buf);
	if ((int) nl_index == -1 || nl_index == len_s_buf - 1)
	{
		ret_s_buf = ft_strdup(s_buf);
		return (ret_s_buf);
	}
	ret_s_buf = str_trim(s_buf, nl_index);
	return (ret_s_buf);
}

/*  *** clean_s_buf (clean static buffer) ***
 *
 *  After a static buffer has been used to find an entire newline,
 *  clean_s_buf will remove an entire newline from the static buffer
 *  to allow carry over to next usage of get_next_line without needing
 *  to read from the file again.
 *  Returns address to new saved carry over on the heap.
 */

static char	*clean_s_buf(char *s_buf)
{
	size_t	nl_index;
	size_t	len_s_buf;
	char	*ret_s_buf;

	nl_index = nl_loc(s_buf);
	if ((int) nl_index == -1)
	{
		free(s_buf);
		return (0);
	}
	len_s_buf = ft_strlen(s_buf);
	if (len_s_buf - nl_index == 1)
	{
		free(s_buf);
		return (0);
	}
	ret_s_buf = ft_strdup(&(s_buf[nl_index + 1]));
	free(s_buf);
	return (ret_s_buf);
}

/*  *** get_next_line (get next line) ***
 *
 *  Uses static char buffer to keep portion after newline
 *  for next function call if needed.
 *  Checks for negative buffer size or invalid file descriptor.
 *  Returns malloced string from staring position to next newline
 *  in open file stream.
 */

char	*get_next_line(int fd)
{
	static char	*s_buf[4096];
	char		*buf;
	char		*ret_str;
	int			r_size;

	if (fd < 0 || BUFFER_SIZE <= 0)
		return (0);
	buf = malloc(BUFFER_SIZE + 1);
	if (!buf)
		return (0);
	r_size = 1;
	while (nl_loc(s_buf[fd]) == -1)
	{
		r_size = read(fd, buf, BUFFER_SIZE);
		if (r_size <= 0)
			break ;
		buf[r_size] = 0;
		s_buf[fd] = ft_strjoin(s_buf[fd], buf);
	}
	free(buf);
	if (r_size == -1)
		return (0);
	ret_str = ret_str_build(s_buf[fd]);
	s_buf[fd] = clean_s_buf(s_buf[fd]);
	return (ret_str);
}


Overwriting ../gnl_working/get_next_line_bonus.c


### get_next_line_utils_bonus.c

In [11]:
%%file ../gnl_working/get_next_line_utils_bonus.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line_utils.c                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:38 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:41 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line_bonus.h"

/*  *** ft_bzero (42 byte zero) ***
 *
 *  Sets memory values to 0 for initilization purposes.
 *  Returns nothing.
 */

void	ft_bzero(void *str, size_t len)
{
	size_t	i;

	i = 0;
	while (i < len)
	{
		*(char *)str = 0;
		str++;
		i++;
	}
}

/*  *** ft_memcpy (42 memory copy) ***
 *
 *  Copies len number of bytes starting from src to dest
 *  Returns pointer to destination string.
 */

void	*ft_memcpy(void *dest, const void *src, size_t len)
{
	size_t	i;

	i = 0;
	if (dest == 0 && src == 0)
		return (0);
	while (i < len)
	{
		*(char *)(dest + i) = *(char *)(src + i);
		i++;
	}
	return (dest);
}

/*  *** ft_strlen (42 string length) ***
 *
 *  Returns number of bytes from str address to next null terminator.
 */

size_t	ft_strlen(const char *str)
{
	size_t	i;

	i = 0;
	while (str && str[i])
		i++;
	return (i);
}

/*  *** ft_strjoin (42 string join) ***
 *
 *  Joins 2 string together to make a new string.
 *  Returns new malloced string 'ptr' that contains s1 followed by s2.
 */

char	*ft_strjoin(char const *s1, char const *s2)
{
	char	*ptr;
	size_t	s1_len;
	size_t	s2_len;

	s1_len = ft_strlen(s1);
	s2_len = ft_strlen(s2);
	if (s1_len + s2_len == 0)
		return (0);
	ptr = (char *)malloc((s1_len + s2_len + 1) * sizeof(char));
	if (ptr == 0)
		return (0);
	ft_bzero(ptr, (s1_len + s2_len + 1));
	ft_memcpy(ptr, (char *)s1, s1_len);
	ft_memcpy((ptr + s1_len), (char *)s2, s2_len);
	free((char *)s1);
	return (ptr);
}

/*  *** ft_strdup (42 string duplicate) ***
 *
 *  Creates a new string that is an exact copy of 'str'.
 *  Returns new malloced string that is an exact copy of 'str'.
 */

char	*ft_strdup(const char *src)
{
	char	*str_ret;
	size_t	src_len;
	size_t	i;

	i = 0;
	src_len = ft_strlen(src);
	str_ret = (char *)malloc((src_len + 1) * sizeof(char));
	if (!str_ret)
		return (0);
	while (src[i])
	{
		str_ret[i] = src[i];
		i++;
	}
	str_ret[i] = '\0';
	return (str_ret);
}


Overwriting ../gnl_working/get_next_line_utils_bonus.c


## Testing

### GNL_MANDATORY_TEST

In [ ]:
%%file gnl_test.c
#include "../gnl_working/get_next_line.h"
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>

int main (void)
{
	char	*string;
	int		i;
//	int		fd1;
	int		fd2;

//	fd1 = open("../gnl_working/tripouille/files/multiple_nlx5", O_RDONLY);
	fd2 = open("sample.txt", O_RDONLY);
	i = 0;
	while (i < 25)
	{
//		string = get_next_line(fd1);
//		if (string)
//			printf("%s", string);
//		free(string);
		string = get_next_line(fd2);
		if (string)
			printf("%s", string);
		free(string);
		i++;
	}
//	close(fd1);
	close(fd2);

	return (0);
}

Overwriting gnl_test.c


In [ ]:
%%bash
SIZE1=42
#SIZE2=1
#rm output*
#cc -Wall -Wextra -Werror -D BUFFER_SIZE=42 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -o gnl_test
#gcc -D BUFFER_SIZE=$SIZE1 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -g -o gnl_test
gcc -Wall -Wextra -Werror -D BUFFER_SIZE=$SIZE1 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -g -o gnl_test
./gnl_test > output1
echo "***output after program***"
cat -vET output1
echo ""
echo "***file direct output***"
cat -vET sample.txt
#cat -vET ../gnl_working/tripouille/files/41_with_nl
# cat output1
#gcc -Wall -Wextra -Werror -D BUFFER_SIZE=$SIZE2 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -g -o gnl_test
#./gnl_test > output2
valgrind -q --leak-check=full --track-origins=yes --read-var-info=yes -s ./gnl_test
#diff output1 output2

***output after program***
This is a sample text.$
Here I will write some shit$
about my program....$
$
lolol ok done.$
$
$
$
$
$
emply lines$

***file direct output***
This is a sample text.$
Here I will write some shit$
about my program....$
$
lolol ok done.$
$
$
$
$
$
emply lines$
This is a sample text.
Here I will write some shit
about my program....

lolol ok done.





emply lines


==195212== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)


### GNL_BONUS_TEST

## Scratch

### File descriptors
- I found out after making this that the struct _IO_FILE is included in the standard library and is used to help interaction with files, but it is not used with read, open, write, close, etc. 
- It is used with fread, fopen, fwrite, and fclose.

In [331]:
%%file fd_examine.h
#ifndef FD_EXAMINE_H_
# define FD_EXAMINE_H_
# include <stdio.h>
# include <unistd.h>

void	print_file_struct_properties(char *label, FILE *p_file);

#endif

Overwriting fd_examine.h


In [332]:
%%file fd_examine.c
#include "fd_examine.h"

void	print_file_struct_properties(char *label, FILE *p_file)
{
	printf("label,size,member type,member name,member description,value,offset from beginning of struct\n");
	printf("whole struct,%ld,,,\n", sizeof(*p_file));
	printf("%s,%d,int,_flags,no description,%d,%ld\n", label, 8, stdin->_flags, (((unsigned long int) &(stdin->_flags)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_ptr,Current read pointer,%p,%ld\n", label, 8, stdin->_IO_read_ptr, (((unsigned long int) &(stdin->_IO_read_ptr)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_end,End of get area,%p,%ld\n", label, 8, stdin->_IO_read_end, (((unsigned long int) &(stdin->_IO_read_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_base,Start of putback+get area,%p,%ld\n", label, 8, stdin->_IO_read_base, (((unsigned long int) &(stdin->_IO_read_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_base,Start of put area,%p,%ld\n", label, 8, stdin->_IO_write_base, (((unsigned long int) &(stdin->_IO_write_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_ptr,Current put pointer,%p,%ld\n", label, 8, stdin->_IO_write_ptr, (((unsigned long int) &(stdin->_IO_write_ptr)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_end,End of put area,%p,%ld\n", label, 8, stdin->_IO_write_end, (((unsigned long int) &(stdin->_IO_write_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_buf_base,Start of reserve area,%p,%ld\n", label, 8, stdin->_IO_buf_base, (((unsigned long int) &(stdin->_IO_buf_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_buf_end,End of reserve area,%p,%ld\n", label, 8, stdin->_IO_buf_end, (((unsigned long int) &(stdin->_IO_buf_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_save_base,Pointer to start of non-current get area,%p,%ld\n", label, 8, stdin->_IO_save_base, (((unsigned long int) &(stdin->_IO_save_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_backup_base,Pointer to first valid character of backup area,%p,%ld\n", label, 8, stdin->_IO_backup_base, (((unsigned long int) &(stdin->_IO_backup_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_save_end,Pointer to end of non-current get area,%p,%ld\n", label, 8, stdin->_IO_save_end, (((unsigned long int) &(stdin->_IO_save_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,struct _IO_marker *,_markers,no description,%p,%ld\n", label, 8, stdin->_markers, (((unsigned long int) &(stdin->_markers)) - ((unsigned long int) stdin)));
	printf("%s,%d,struct _IO_FILE *,_chain,no description,%p,%ld\n", label, 8, stdin->_chain, (((unsigned long int) &(stdin->_chain)) - ((unsigned long int) stdin)));
	printf("%s,%d,int,_fileno,no description,%d,%ld\n", label, 4, stdin->_fileno, (((unsigned long int) &(stdin->_fileno)) - ((unsigned long int) stdin)));
	//printf("%s,%d,int,_blksize,no description,%d,%ld\n", label, 4, stdin->_blksize, (((unsigned long int) &(stdin->_blksize)) - ((unsigned long int) stdin)));
	printf("%s,%d,int,_flags2,no description,%d,%ld\n", label, 4, stdin->_flags2, (((unsigned long int) &(stdin->_flags2)) - ((unsigned long int) stdin)));
	printf("%s,%d,_IO_off_t,_old_offset,This used to be _offset but it's too small,%ld,%ld\n", label, 8, stdin->_old_offset, (((unsigned long int) &(stdin->_old_offset)) - ((unsigned long int) stdin)));
	printf("%s,%d,unsigned short,_cur_column,no description,%d,%ld\n", label, 2, stdin->_cur_column, (((unsigned long int) &(stdin->_cur_column)) - ((unsigned long int) stdin)));
	printf("%s,%d,signed char,_vtable_offset,no description,%d,%ld\n", label, 8, stdin->_vtable_offset, (((unsigned long int) &(stdin->_vtable_offset)) - ((unsigned long int) stdin)));
	printf("%s,%d,char,_shortbuf,no description,%s,%ld\n", label, 8, stdin->_shortbuf, (((unsigned long int) &(stdin->_shortbuf)) - ((unsigned long int) stdin)));
	printf("%s,%d,_IO_lock_t *,_lock,no description,%p,%ld\n", label, 8, stdin->_lock, (((unsigned long int) &(stdin->_lock)) - ((unsigned long int) stdin)));

	return ;
}

Overwriting fd_examine.c


In [333]:
%%file fd_test.c
#include <stdio.h>
#include <unistd.h>
#include "fd_examine.h"

int main (void)
{
	//printf("i/o    :descriptor\tpointer\t\t\n");
	//printf("stdin  :%d\t\t%p\n", STDIN_FILENO, stdin);
	//printf("stdout :%d\t\t%p\n", STDOUT_FILENO, stdout);
	// printf("stderr :%d\t\t%p\n", STDERR_FILENO, stderr);

	print_file_struct_properties("stdin", stdin);
	//print_file_struct_properties("stdout", stdout);
	//print_file_struct_properties("stderr", stderr);

	return (0);
}

Overwriting fd_test.c


In [334]:
%%bash
gcc fd_test.c fd_examine.c -g
./a.out > output.csv

### Stack Frame
- This is a simple program for me to use with gdb to view the stack frame.

In [24]:
%%file stack_frame.c
#include <stdlib.h>

char	*return_string(void)
{
	char	*str;
	str = (char *)malloc(10);
	return (str);
}

int foo (int n)
{
	static char *here;
	here = 0;
	here = return_string();
	return (n + 1);
}

int main (void)
{
	int	n;

	n = 0;
	n = foo(n);
	return (0);
}

Overwriting stack_frame.c


In [25]:
%%bash
gcc stack_frame.c -g -o stack_frame
./stack_frame
valgrind --leak-check=full --show-leak-kinds=all -s ./stack_frame

==62578== Memcheck, a memory error detector
==62578== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==62578== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==62578== Command: ./stack_frame
==62578== 
==62578== 
==62578== HEAP SUMMARY:
==62578==     in use at exit: 10 bytes in 1 blocks
==62578==   total heap usage: 1 allocs, 0 frees, 10 bytes allocated
==62578== 
==62578== 10 bytes in 1 blocks are still reachable in loss record 1 of 1
==62578==    at 0x4848899: malloc (in /usr/libexec/valgrind/vgpreload_memcheck-amd64-linux.so)
==62578==    by 0x10915E: return_string (stack_frame.c:6)
==62578==    by 0x109187: foo (stack_frame.c:14)
==62578==    by 0x1091B3: main (stack_frame.c:23)
==62578== 
==62578== LEAK SUMMARY:
==62578==    definitely lost: 0 bytes in 0 blocks
==62578==    indirectly lost: 0 bytes in 0 blocks
==62578==      possibly lost: 0 bytes in 0 blocks
==62578==    still reachable: 10 bytes in 1 blocks
==62578==         suppressed: 0 byt